### Workbook for comparing population counts for villages surrounding Rwanda bridges
Weeks of April 28, 2025 |
Author: Adele Birkenes

In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import rasterio
from rasterio.merge import merge
from rasterio.io import MemoryFile
import os
import numpy as np

# Import pop-estimates notebook
from importnb import Notebook
with Notebook():
    from pop_estimates import reproject_raster, extract_pop

Task 1: Set synced & unsynced file paths, read in needs assessment data, set CRS, convert data type to numeric, and filter out null population values

In [4]:
synced_path = "../../synced-data/population-exploration/"
unsynced_path = "../../unsynced-data"

# Specify file path for needs assessment data
needs_assessment_fp = os.path.join(unsynced_path, "processed_needs_assessments_05_11.geojson") #update if needed

# Import original village boundaries so that the correct CRS can be set
village_boundaries_fp = os.path.join(synced_path, "Rwanda Village Boundaries/Village.shp")
village_boundaries = gpd.read_file(village_boundaries_fp)

# Read the needs assessment data into a geodataframe and set the CRS to that of the village boundaries file
needs_assessment = gpd.read_file(needs_assessment_fp).set_crs(village_boundaries.crs, allow_override=True)
print(needs_assessment.crs)

PROJCS["TM_Rwanda",GEOGCS["ITRF2005",DATUM["International_Terrestrial_Reference_Frame_2005",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6896"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",30],PARAMETER["scale_factor",0.9999],PARAMETER["false_easting",500000],PARAMETER["false_northing",5000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [ ]:
# The below code block is commented out because it is no longer needed
# This is because the needs-assessment.ipynb included steps to ensure values are numeric and filter out null values

# # Print data type of "Individuals Served" column
# print(needs_assessment['Individuals Served'].dtype)

# # The data type is "object," meaning that it contains non-numeric values
# # Therefore, convert the "Individuals Served" column to numeric, coercing errors to NaN
# needs_assessment["Individuals Served"] = pd.to_numeric(needs_assessment["Individuals Served"], errors='coerce')

# # Filter out rows with NaN values in the "Individuals Served" column
# needs_assessment = needs_assessment[needs_assessment["Individuals Served"].notna()]

# # Reset index of geodataframe
# needs_assessment.reset_index(drop=True, inplace=True)

# # Confirm numeric data type and print first 3 rows of needs assessment data
# print(needs_assessment['Individuals Served'].dtype)
# print(needs_assessment.head(3))

# # Map needs assessment data
# # needs_assessment.explore()

float64
float64
      Admin 2 Village_ID Cell_ID Sector_ID Distr_ID Prov_ID      Admin 5  \
0  Nyarugenge   11010103  110101      1101       11       1   Iterambere   
1  Nyarugenge   11010106  110101      1101       11       1     Nyenyeri   
2  Nyarugenge   11010107  110101      1101       11       1  Ubukorikori   

   Admin 4_x Admin 3                     Admin 1_x  \
0  Akabahizi  Gitega  Kigali Town/Umujyi wa Kigali   
1  Akabahizi  Gitega  Kigali Town/Umujyi wa Kigali   
2  Akabahizi  Gitega  Kigali Town/Umujyi wa Kigali   

  Bridge Opportunity: CaseSafeID_x Bridge Opportunity: Project Code  \
0               006f100000eekXgAAI                          1014263   
1               006f100000eekXgAAI                          1014263   
2               006f100000eekXCAAY                          1014258   

  Bridge Opportunity: Country Bridge Opportunity: Opportunity Name  \
0                      Rwanda        Rwanda - Iterambere - 1014263   
1                      Rwanda        

Task 2: Read in gridded population data and use imported extract_pop function to extract population counts for each needs assessment community

In [4]:
# WorldPop
rwa_pop_2020 = os.path.join(synced_path, "rwa_ppp_2020.tif")

# Landscan
landscan_2018 = os.path.join(unsynced_path, "landscan-global-2018-assets/landscan-global-2018.tif")

In [5]:
# GHS-POP
GHS_POP_2020 = [os.path.join(unsynced_path, "GHS_POP_E2020_GLOBE_R2023A_4326_3ss_V1_0_R10_C21.tif"),
                os.path.join(unsynced_path, "GHS_POP_E2020_GLOBE_R2023A_4326_3ss_V1_0_R10_C22.tif")]

# Mosaic two GHS-POP rasters to cover extent of Rwanda
GHS_POP_2020_mosaic = merge([rasterio.open(raster) for raster in GHS_POP_2020])

In [6]:
# Extract population estimates from rasters for each admin 5 polygon in the needs assessment data
## WorldPop
pop_estimates_comparison = extract_pop(population_raster_file=rwa_pop_2020,
                                       population_raster_name="WorldPop_2020",
                                       villages_near_bridges=needs_assessment)

## Landscan
pop_estimates_comparison = extract_pop(population_raster_file=landscan_2018,
                                       population_raster_name="Landscan_2018",
                                       villages_near_bridges=pop_estimates_comparison)

## GHS-POP
# Save the GHS_POP_2020_mosaic to a temporary file
temp_raster_path = os.path.join(unsynced_path, "temp_GHS_POP_2020.tif")
with rasterio.open(
    temp_raster_path,
    'w',
    driver='GTiff',
    height=GHS_POP_2020_mosaic[0].shape[1],
    width=GHS_POP_2020_mosaic[0].shape[2],
    count=1,
    dtype=GHS_POP_2020_mosaic[0].dtype.name,
    crs='EPSG:4326',
    transform=GHS_POP_2020_mosaic[1]
) as dataset:
    dataset.write(GHS_POP_2020_mosaic[0][0], 1)

# Run extract_pop function on the temporary GHS_POP_2020 raster
pop_estimates_comparison = extract_pop(population_raster_file=temp_raster_path,
                                       population_raster_name="GHS_POP_2020",
                                       villages_near_bridges=pop_estimates_comparison)

# Remove the temporary raster file after extraction
os.remove(temp_raster_path)

pop_estimates_comparison.head()

CRS: EPSG:4326
Raster shape: (2152, 2447)
Number of Bands: 1
CRS: EPSG:4326
Raster shape: (21600, 43200)
Number of Bands: 1
CRS: EPSG:4326
Raster shape: (12000, 24000)
Number of Bands: 1


,Admin 2,Village_ID,Cell_ID,Sector_ID,Distr_ID,Prov_ID,Admin 5,Admin 4_x,Admin 3,Admin 1_x,Bridge Opportunity: CaseSafeID_x,Bridge Opportunity: Project Code,Bridge Opportunity: Country,Bridge Opportunity: Opportunity Name,Bridge Opportunity: Stage,Bridge Opportunity: Sub-Stage,Bridge Opportunity: Close Date,Admin 1_y,Admin 4_y,Bridge Name,Bridge Opportunity: River Name,Bridge Opportunity: Research Initiative,Bridge Opportunity: Monitoring Campaign,Form: Form Name,Bridge Opportunity: GPS (Latitude),Bridge Opportunity: GPS (Longitude),Individuals Directly Served,Bridge Opportunity: Population Dataset URL,Bridge Opportunity: Population Estimate 2000m,Row ID,Community Column,Community Number,Bridge Opportunity: CaseSafeID_y,Individuals Served Column,Individuals Served,geometry,WorldPop_2020,Landscan_2018,GHS_POP_2020
0,Nyarugenge,11010103,110101,1101,11,1,Iterambere,Akabahizi,Gitega,Kigali Town/Umujyi wa Kigali,006f100000eekXgAAI,1014263,Rwanda,Rwanda - Iterambere - 1014263,Rejected,Technical,11/17/18,Kigali,Akabahizi,Iterambere,Ruhurura,None,None,Project Assessment - 2018.11.11,-1.945440,30.05341,NaN,http://www.worldpop.org.uk/data/summary?doi=10...,172079.0,1257.0,Community Served 1,1,006f100000eekXgAAI,Individuals served in Community 1,510.0,"POLYGON Z ((30.05147 -1.94394 0, 30.05169 -1.9...",1927.160400,16697,1257.567639
1,Nyarugenge,11010106,110101,1101,11,1,Nyenyeri,Akabahizi,Gitega,Kigali Town/Umujyi wa Kigali,006f100000eekXgAAI,1014263,Rwanda,Rwanda - Iterambere - 1014263,Rejected,Technical,11/17/18,Kigali,Akabahizi,Iterambere,Ruhurura,None,None,Project Assessment - 2018.11.11,-1.945440,30.05341,NaN,http://www.worldpop.org.uk/data/summary?doi=10...,172079.0,1257.0,Community Served 2,2,006f100000eekXgAAI,Individuals served in Community 2,652.0,"POLYGON Z ((30.05195 -1.94681 0, 30.05203 -1.9...",535.933838,0,373.604518
2,Nyarugenge,11010107,110101,1101,11,1,Ubukorikori,Akabahizi,Gitega,Kigali Town/Umujyi wa Kigali,006f100000eekXCAAY,1014258,Rwanda,Rwanda - Ubukorikori - 1014258,Rejected,Technical,11/30/18,Kigali,Kora,Ubukorikori,Ruhurura,None,None,Project Assessment - 2018.11.11,-1.949340,30.05571,NaN,http://www.worldpop.org.uk/data/summary?doi=10...,190281.0,1253.0,Community Served 2,2,006f100000eekXCAAY,Individuals served in Community 2,523.0,"POLYGON Z ((30.05488 -1.94678 0, 30.05496 -1.9...",1625.287109,0,1522.424386
3,Nyarugenge,11010108,110101,1101,11,1,Ubumwe,Akabahizi,Gitega,Kigali Town/Umujyi wa Kigali,006f100000eekWsAAI,1014254,Rwanda,Rwanda - Ubumwe - 1014254,Rejected,Technical,11/17/18,Kigali,Gacyamo,Ubumwe,Mpazi,None,None,Project Assessment - 2018.11.11,-1.966149,30.05606,NaN,http://www.worldpop.org.uk/data/summary?doi=10...,193056.0,1250.0,Community Served 2,2,006f100000eekWsAAI,Individuals served in Community 2,523.0,"POLYGON Z ((30.04829 -1.94176 0, 30.04833 -1.9...",1419.238159,0,1042.088431
4,Nyarugenge,11010108,110101,1101,11,1,Ubumwe,Akabahizi,Gitega,Kigali Town/Umujyi wa Kigali,006f100000eekX7AAI,1014257,Rwanda,Rwanda - Ituze - 1014257,Rejected,Technical,12/19/18,Kigali,Akabeza,Ituze,Ruhurura,None,None,Project Assessment - 2018.11.11,-1.953220,30.05520,NaN,http://www.worldpop.org.uk/data/summary?doi=10...,207606.0,1252.0,Community Served 2,2,006f100000eekX7AAI,Individuals served in Community 2,524.0,"POLYGON Z ((30.04829 -1.94176 0, 30.04833 -1.9...",1419.238159,0,1042.088431
